##📦 instance_generator.py

In [ ]:
import numpy as np
import random

class CMgInstance:
    def __init__(self, n_tasks=10, n_providers=3):
        self.n_tasks = n_tasks
        self.n_providers = n_providers

        # ---------- GENERATE STRUCTURE ----------
        self.subtasks_per_task = [random.randint(2,5) for _ in range(n_tasks)]
        self.machines_per_provider = [random.randint(2,5) for _ in range(n_providers)]

        # ---------- ARRIVAL TIMES ----------
        self.arrival_time = np.random.randint(0,200, size=n_tasks)

        # ---------- PROCESSING TIMES ----------
        # P[i][j][k][r]
        self.processing_time = self.generate_processing_times()

        # ---------- LOGISTICS ----------
        self.TSS = np.random.randint(5,30,(n_providers,n_providers))  # SP to SP
        self.TSC = np.random.randint(5,50,(n_providers,n_tasks))      # SP to customer

    # -----------------------------------------
    def generate_processing_times(self):
        P = {}
        for i in range(self.n_tasks):
            P[i] = {}
            for j in range(self.subtasks_per_task[i]):
                P[i][j] = {}
                for k in range(self.n_providers):
                    P[i][j][k] = {}
                    for r in range(self.machines_per_provider[k]):
                        P[i][j][k][r] = random.randint(10,100)
        return P

# ----------- helper to create benchmark sizes ----------
def generate_instance(size="small"):

    if size == "small":
        return CMgInstance(n_tasks=10, n_providers=3)
    elif size == "medium":
        return CMgInstance(n_tasks=20, n_providers=5)
    else:
        return CMgInstance(n_tasks=35, n_providers=8)

##File.Test

In [ ]:
module_content = """import numpy as np
import random

class CMgInstance:
    def __init__(self, n_tasks=10, n_providers=3):
        self.n_tasks = n_tasks
        self.n_providers = n_providers

        # ---------- GENERATE STRUCTURE ----------
        self.subtasks_per_task = [random.randint(2,5) for _ in range(n_tasks)]
        self.machines_per_provider = [random.randint(2,5) for _ in range(n_providers)]

        # ---------- ARRIVAL TIMES ----------
        self.arrival_time = np.random.randint(0,200, size=n_tasks)

        # ---------- PROCESSING TIMES ----------
        # P[i][j][k][r]
        self.processing_time = self.generate_processing_times()

        # ---------- LOGISTICS ----------
        self.TSS = np.random.randint(5,30,(n_providers,n_providers))  # SP to SP
        self.TSC = np.random.randint(5,50,(n_providers,n_tasks))      # SP to customer

    # -----------------------------------------
    def generate_processing_times(self):
        P = {}
        for i in range(self.n_tasks):
            P[i] = {}
            for j in range(self.subtasks_per_task[i]):
                P[i][j] = {}
                for k in range(self.n_providers):
                    P[i][j][k] = {}
                    for r in range(self.machines_per_provider[k]):
                        P[i][j][k][r] = random.randint(10,100)
        return P

# ----------- helper to create benchmark sizes ----------
def generate_instance(size="small"):

    if size == "small":
        return CMgInstance(n_tasks=10, n_providers=3)
    elif size == "medium":
        return CMgInstance(n_tasks=20, n_providers=5)
    else:
        return CMgInstance(n_tasks=35, n_providers=8)
"""

with open('instance_generator.py', 'w') as f:
    f.write(module_content)

from instance_generator import generate_instance

inst = generate_instance("small")

print("Tasks:", inst.n_tasks)
print("Providers:", inst.n_providers)
print("Subtasks per task:", inst.subtasks_per_task)
print("Arrival time:", inst.arrival_time)

##📦 scheduler_simulator.py

In [ ]:
import numpy as np
from collections import defaultdict

class Scheduler:
    def __init__(self, instance):
        self.inst = instance

    # --------------------------------------------------
    # انتخاب بهترین ماشین برای یک عملیات
    # --------------------------------------------------
    def select_best_machine(self, task, subtask, ready_time):
        best_finish = 1e18
        best_choice = None

        for k in range(self.inst.n_providers):
            for r in range(self.inst.machines_per_provider[k]):

                proc_time = self.inst.processing_time[task][subtask][k][r]
                finish_time = ready_time + proc_time

                if finish_time < best_finish:
                    best_finish = finish_time
                    best_choice = (k,r,proc_time)

        return best_choice  # provider, machine, processing_time

    # --------------------------------------------------
    # DECODE chromosome → schedule → makespan
    # --------------------------------------------------
    def evaluate(self, chromosome):

        # زمان آزاد شدن ماشین‌ها
        machine_available = defaultdict(int)

        # پایان آخرین subtask هر task
        task_last_finish = defaultdict(int)

        # شماره subtask بعدی که باید اجرا شود
        next_subtask_index = defaultdict(int)

        # شروع کار = arrival time
        for t in range(self.inst.n_tasks):
            task_last_finish[t] = self.inst.arrival_time[t]

        # ----------------------------------------------
        for gene in chromosome:
            task = gene

            subtask = next_subtask_index[task]

            # اگر همه subtasks انجام شده → رد کن
            if subtask >= self.inst.subtasks_per_task[task]:
                continue

            ready_time = task_last_finish[task]

            # انتخاب بهترین ماشین
            provider, machine, proc_time = self.select_best_machine(
                task, subtask, ready_time
            )

            machine_key = (provider, machine)
            start_time = max(machine_available[machine_key], ready_time)
            finish_time = start_time + proc_time

            # آپدیت وضعیت‌ها
            machine_available[machine_key] = finish_time
            task_last_finish[task] = finish_time
            next_subtask_index[task] += 1

        # ----------------------------------------------
        # makespan = آخرین پایان
        makespan = max(task_last_finish.values())
        return makespan

##Test

In [ ]:
import random

# Create scheduler_simulator.py
module_content_scheduler = """import numpy as np
from collections import defaultdict

class Scheduler:
    def __init__(self, instance):
        self.inst = instance

    # --------------------------------------------------
    # انتخاب بهترین ماشین برای یک عملیات
    # --------------------------------------------------
    def select_best_machine(self, task, subtask, ready_time):
        best_finish = 1e18
        best_choice = None

        for k in range(self.inst.n_providers):
            for r in range(self.inst.machines_per_provider[k]):

                proc_time = self.inst.processing_time[task][subtask][k][r]
                finish_time = ready_time + proc_time

                if finish_time < best_finish:
                    best_finish = finish_time
                    best_choice = (k,r,proc_time)

        return best_choice  # provider, machine, processing_time

    # --------------------------------------------------
    # DECODE chromosome → schedule → makespan
    # --------------------------------------------------
    def evaluate(self, chromosome):

        # زمان آزاد شدن ماشین‌ها
        machine_available = defaultdict(int)

        # پایان آخرین subtask هر task
        task_last_finish = defaultdict(int)

        # شماره subtask بعدی که باید اجرا شود
        next_subtask_index = defaultdict(int)

        # شروع کار = arrival time
        for t in range(self.inst.n_tasks):
            task_last_finish[t] = self.inst.arrival_time[t]

        # ----------------------------------------------
        for gene in chromosome:
            task = gene

            subtask = next_subtask_index[task]

            # اگر همه subtasks انجام شده → رد کن
            if subtask >= self.inst.subtasks_per_task[task]:
                continue

            ready_time = task_last_finish[task]

            # انتخاب بهترین ماشین
            provider, machine, proc_time = self.select_best_machine(
                task, subtask, ready_time
            )

            machine_key = (provider, machine)
            start_time = max(machine_available[machine_key], ready_time)
            finish_time = start_time + proc_time

            # آپدیت وضعیت‌ها
            machine_available[machine_key] = finish_time
            task_last_finish[task] = finish_time
            next_subtask_index[task] += 1

        # ----------------------------------------------
        # makespan = آخرین پایان
        makespan = max(task_last_finish.values())
        return makespan"""

with open('scheduler_simulator.py', 'w') as f:
    f.write(module_content_scheduler)

from instance_generator import generate_instance
from scheduler_simulator import Scheduler

inst = generate_instance("small")
scheduler = Scheduler(inst)

# ساخت یک chromosome تصادفی
chromosome = []
for t in range(inst.n_tasks):
    chromosome += [t] * inst.subtasks_per_task[t]

random.shuffle(chromosome)

makespan = scheduler.evaluate(chromosome)
print("Makespan:", makespan)

##🧬 Step 3 — Genetic Algorithm Baseline

In [ ]:
# Initialize population
# Repeat for generations:
#     Selection
#     Crossover
#     Mutation
#     Evaluation
#     Elitism
# Return best solution

##📦 File operators.py

In [ ]:
import random

# -----------------------------
# MUTATIONS
# -----------------------------
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

def pairwise_swap(chrom):
    for i in range(0, len(chrom)-1, 2):
        if random.random() < 0.5:
            chrom[i], chrom[i+1] = chrom[i+1], chrom[i]
    return chrom

def inversion_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    chrom[a:b] = reversed(chrom[a:b])
    return chrom

# -----------------------------
# CROSSOVERS
# -----------------------------
def one_point_crossover(p1, p2):
    cut = random.randint(1, len(p1)-2)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def two_point_crossover(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    c1 = p1[:a] + p2[a:b] + p1[b:]
    c2 = p2[:a] + p1[a:b] + p2[b:]
    return c1, c2

def uniform_crossover(p1, p2):
    c1, c2 = [], []
    for g1, g2 in zip(p1, p2):
        if random.random() < 0.5:
            c1.append(g1); c2.append(g2)
        else:
            c1.append(g2); c2.append(g1)
    return c1, c2



##📦 File ga_baseline.py

In [ ]:
import random
import numpy as np
from scheduler_simulator import Scheduler

# Create operators.py
module_content_operators = """import random

# -----------------------------
# MUTATIONS
# -----------------------------
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

def pairwise_swap(chrom):
    for i in range(0, len(chrom)-1, 2):
        if random.random() < 0.5:
            chrom[i], chrom[i+1] = chrom[i+1], chrom[i]
    return chrom

def inversion_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    chrom[a:b] = reversed(chrom[a:b])
    return chrom

# -----------------------------
# CROSSOVERS
# -----------------------------
def one_point_crossover(p1, p2):
    cut = random.randint(1, len(p1)-2)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def two_point_crossover(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    c1 = p1[:a] + p2[a:b] + p1[b:]
    c2 = p2[:a] + p1[a:b] + p2[b:]
    return c1, c2

def uniform_crossover(p1, p2):
    c1, c2 = [], []
    for g1, g2 in zip(p1, p2):
        if random.random() < 0.5:
            c1.append(g1); c2.append(g2)
        else:
            c1.append(g2); c2.append(g1)
    return c1, c2"""

with open('operators.py', 'w') as f:
    f.write(module_content_operators)

from operators import *

class GeneticAlgorithm:
    def __init__(self, instance, pop_size=40, generations=60):
        self.inst = instance
        self.scheduler = Scheduler(instance)
        self.pop_size = pop_size
        self.generations = generations

        self.mutations = [swap_mutation, pairwise_swap, inversion_mutation]
        self.crossovers = [one_point_crossover, two_point_crossover, uniform_crossover]

    # ------------------------------------------
    def create_chromosome(self):
        chrom = []
        for t in range(self.inst.n_tasks):
            chrom += [t] * self.inst.subtasks_per_task[t]
        random.shuffle(chrom)
        return chrom

    # ------------------------------------------
    def init_population(self):
        return [self.create_chromosome() for _ in range(self.pop_size)]

    # ------------------------------------------
    def evaluate_population(self, pop):
        return [self.scheduler.evaluate(ind) for ind in pop]

    # ------------------------------------------
    def tournament_selection(self, pop, fitness, k=3):
        selected = random.sample(range(len(pop)), k)
        best = min(selected, key=lambda i: fitness[i])
        return pop[best]

    # ------------------------------------------
    def run(self):
        population = self.init_population()
        best_global = None
        best_fit = 1e18

        for gen in range(self.generations):
            fitness = self.evaluate_population(population)

            # elitism
            elite_idx = np.argmin(fitness)
            elite = population[elite_idx]

            if fitness[elite_idx] < best_fit:
                best_fit = fitness[elite_idx]
                best_global = elite

            new_pop = [elite]

            while len(new_pop) < self.pop_size:
                p1 = self.tournament_selection(population, fitness)
                p2 = self.tournament_selection(population, fitness)

                crossover = random.choice(self.crossovers)
                c1, c2 = crossover(p1.copy(), p2.copy())

                mutation = random.choice(self.mutations)
                c1 = mutation(c1)
                c2 = mutation(c2)

                new_pop += [c1, c2]

            population = new_pop[:self.pop_size]
            print(f"Gen {gen}  Best = {best_fit}")

        return best_global, best_fit

##� Test GA

In [ ]:
# module_content for instance_generator.py
module_content_instance = """import numpy as np
import random

class CMgInstance:
    def __init__(self, n_tasks=10, n_providers=3):
        self.n_tasks = n_tasks
        self.n_providers = n_providers

        # ---------- GENERATE STRUCTURE ----------
        self.subtasks_per_task = [random.randint(2,5) for _ in range(n_tasks)]
        self.machines_per_provider = [random.randint(2,5) for _ in range(n_providers)]

        # ---------- ARRIVAL TIMES ----------
        self.arrival_time = np.random.randint(0,200, size=n_tasks)

        # ---------- PROCESSING TIMES ----------
        # P[i][j][k][r]
        self.processing_time = self.generate_processing_times()

        # ---------- LOGISTICS ----------
        self.TSS = np.random.randint(5,30,(n_providers,n_providers))  # SP to SP
        self.TSC = np.random.randint(5,50,(n_providers,n_tasks))      # SP to customer

    # -----------------------------------------
    def generate_processing_times(self):
        P = {}
        for i in range(self.n_tasks):
            P[i] = {}
            for j in range(self.subtasks_per_task[i]):
                P[i][j] = {}
                for k in range(self.n_providers):
                    P[i][j][k] = {}
                    for r in range(self.machines_per_provider[k]):
                        P[i][j][k][r] = random.randint(10,100)
        return P

# ----------- helper to create benchmark sizes ----------
def generate_instance(size="small"):

    if size == "small":
        return CMgInstance(n_tasks=10, n_providers=3)
    elif size == "medium":
        return CMgInstance(n_tasks=20, n_providers=5)
    else:
        return CMgInstance(n_tasks=35, n_providers=8)
"""
with open('instance_generator.py', 'w') as f:
    f.write(module_content_instance)


# module_content for scheduler_simulator.py
module_content_scheduler = """import numpy as np
from collections import defaultdict

class Scheduler:
    def __init__(self, instance):
        self.inst = instance

    # --------------------------------------------------
    # انتخاب بهترین ماشین برای یک عملیات
    # --------------------------------------------------
    def select_best_machine(self, task, subtask, ready_time):
        best_finish = 1e18
        best_choice = None

        for k in range(self.inst.n_providers):
            for r in range(self.inst.machines_per_provider[k]):

                proc_time = self.inst.processing_time[task][subtask][k][r]
                finish_time = ready_time + proc_time

                if finish_time < best_finish:
                    best_finish = finish_time
                    best_choice = (k,r,proc_time)

        return best_choice  # provider, machine, processing_time

    # --------------------------------------------------
    # DECODE chromosome → schedule → makespan
    # --------------------------------------------------
    def evaluate(self, chromosome):

        # زمان آزاد شدن ماشین‌ها
        machine_available = defaultdict(int)

        # پایان آخرین subtask هر task
        task_last_finish = defaultdict(int)

        # شماره subtask بعدی که باید اجرا شود
        next_subtask_index = defaultdict(int)

        # شروع کار = arrival time
        for t in range(self.inst.n_tasks):
            task_last_finish[t] = self.inst.arrival_time[t]

        # ----------------------------------------------
        for gene in chromosome:
            task = gene

            subtask = next_subtask_index[task]

            # اگر همه subtasks انجام شده → رد کن
            if subtask >= self.inst.subtasks_per_task[task]:
                continue

            ready_time = task_last_finish[task]

            # انتخاب بهترین ماشین
            provider, machine, proc_time = self.select_best_machine(
                task, subtask, ready_time
            )

            machine_key = (provider, machine)
            start_time = max(machine_available[machine_key], ready_time)
            finish_time = start_time + proc_time

            # آپدیت وضعیت‌ها
            machine_available[machine_key] = finish_time
            task_last_finish[task] = finish_time
            next_subtask_index[task] += 1

        # ----------------------------------------------
        # makespan = آخرین پایان
        makespan = max(task_last_finish.values())
        return makespan
"""
with open('scheduler_simulator.py', 'w') as f:
    f.write(module_content_scheduler)


# module_content for operators.py
module_content_operators = """import random

# -----------------------------
# MUTATIONS
# -----------------------------
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

def pairwise_swap(chrom):
    for i in range(0, len(chrom)-1, 2):
        if random.random() < 0.5:
            chrom[i], chrom[i+1] = chrom[i+1], chrom[i]
    return chrom

def inversion_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    chrom[a:b] = reversed(chrom[a:b])
    return chrom

# -----------------------------
# CROSSOVERS
# -----------------------------
def one_point_crossover(p1, p2):
    cut = random.randint(1, len(p1)-2)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def two_point_crossover(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    c1 = p1[:a] + p2[a:b] + p1[b:]
    c2 = p2[:a] + p1[a:b] + p2[b:]
    return c1, c2

def uniform_crossover(p1, p2):
    c1, c2 = [], []
    for g1, g2 in zip(p1, p2):
        if random.random() < 0.5:
            c1.append(g1); c2.append(g2)
        else:
            c1.append(g2); c2.append(g1)
    return c1, c2
"""
with open('operators.py', 'w') as f:
    f.write(module_content_operators)


# module_content for ga_baseline.py
# Note: This is an adjusted version of the content of cell 1nl8TvXV4wDZ
# without the internal 'operators.py' creation logic, as that's handled above.
module_content_ga_baseline = """import random
import numpy as np
from scheduler_simulator import Scheduler
from operators import *

class GeneticAlgorithm:
    def __init__(self, instance, pop_size=40, generations=60):
        self.inst = instance
        self.scheduler = Scheduler(instance)
        self.pop_size = pop_size
        self.generations = generations

        self.mutations = [swap_mutation, pairwise_swap, inversion_mutation]
        self.crossovers = [one_point_crossover, two_point_crossover, uniform_crossover]

    # ------------------------------------------
    def create_chromosome(self):
        chrom = []
        for t in range(self.inst.n_tasks):
            chrom += [t] * self.inst.subtasks_per_task[t]
        random.shuffle(chrom)
        return chrom

    # ------------------------------------------
    def init_population(self):
        return [self.create_chromosome() for _ in range(self.pop_size)]

    # ------------------------------------------
    def evaluate_population(self, pop):
        return [self.scheduler.evaluate(ind) for ind in pop]

    # ------------------------------------------
    def tournament_selection(self, pop, fitness, k=3):
        selected = random.sample(range(len(pop)), k)
        best = min(selected, key=lambda i: fitness[i])
        return pop[best]

    # ------------------------------------------
    def run(self):
        population = self.init_population()
        best_global = None
        best_fit = 1e18

        for gen in range(self.generations):
            fitness = self.evaluate_population(population)

            # elitism
            elite_idx = np.argmin(fitness)
            elite = population[elite_idx]

            if fitness[elite_idx] < best_fit:
                best_fit = fitness[elite_idx]
                best_global = elite

            new_pop = [elite]

            while len(new_pop) < self.pop_size:
                p1 = self.tournament_selection(population, fitness)
                p2 = self.tournament_selection(population, fitness)

                crossover = random.choice(self.crossovers)
                c1, c2 = crossover(p1.copy(), p2.copy())

                mutation = random.choice(self.mutations)
                c1 = mutation(c1)
                c2 = mutation(c2)

                new_pop += [c1, c2]

            population = new_pop[:self.pop_size]
            print(f"Gen {gen}  Best = {best_fit}")

        return best_global, best_fit
"""
with open('ga_baseline.py', 'w') as f:
    f.write(module_content_ga_baseline)


from instance_generator import generate_instance
from ga_baseline import GeneticAlgorithm

inst = generate_instance("small")

ga = GeneticAlgorithm(inst)
best_sol, best_fit = ga.run()

print("Final Makespan:", best_fit)


##🤖 Step 4 — Q-Learning module (مغز QLGA)

In [ ]:
prev_best = makespan
current_best = best_fit
improvement = prev_best - current_best

print(f"Initial makespan (random chromosome): {prev_best}")
print(f"GA-optimized makespan: {current_best}")
print(f"Improvement: {improvement}")

In [ ]:
if improvement > 20:
    state = 0
elif 0 < improvement <= 20:
    state = 1
else:
    state = 2

print(f"Improvement: {improvement}, State: {state}")

##Actions

0 swap mutation
1 pairwise swap
2 inversion
3 one-point crossover
4 two-point crossover
5 uniform crossover

##📦 q_learning.p

In [ ]:
import numpy as np
import random

class QLearningAgent:
    def __init__(self, n_states=3, n_actions=6,
                 alpha=0.1, gamma=0.9, epsilon=0.2):

        self.Q = np.zeros((n_states, n_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    # ---------------------------------
    def get_state(self, improvement):
        if improvement > 20:
            return 0
        elif improvement > 0:
            return 1
        else:
            return 2

    # ---------------------------------
    def choose_action(self, state):
        # ε-greedy
        if random.random() < self.epsilon:
            return random.randint(0,5)
        return np.argmax(self.Q[state])

    # ---------------------------------
    def update(self, state, action, reward, next_state):
        best_next = np.max(self.Q[next_state])

        self.Q[state, action] += self.alpha * (
            reward + self.gamma * best_next - self.Q[state, action]
        )

##🏆 Step 5 — پیاده‌سازی QLGA (الگوریتم پیشنهادی مقاله)

##📦 File qlga.py

In [ ]:
import random
import numpy as np

# Create q_learning.py
module_content_q_learning = """import numpy as np
import random

class QLearningAgent:
    def __init__(self, n_states=3, n_actions=6,
                 alpha=0.1, gamma=0.9, epsilon=0.2):

        self.Q = np.zeros((n_states, n_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    # ---------------------------------
    def get_state(self, improvement):
        if improvement > 20:
            return 0
        elif improvement > 0:
            return 1
        else:
            return 2

    # ---------------------------------
    def choose_action(self, state):
        # \u03b5-greedy
        if random.random() < self.epsilon:
            return random.randint(0,5)
        return np.argmax(self.Q[state])

    # ---------------------------------
    def update(self, state, action, reward, next_state):
        best_next = np.max(self.Q[next_state])

        self.Q[state, action] += self.alpha * (
            reward + self.gamma * best_next - self.Q[state, action]
        )"""

with open('q_learning.py', 'w') as f:
    f.write(module_content_q_learning)


from scheduler_simulator import Scheduler
from operators import *
from q_learning import QLearningAgent

class QLGA:
    def __init__(self, instance, pop_size=40, generations=60):
        self.inst = instance
        self.scheduler = Scheduler(instance)
        self.pop_size = pop_size
        self.generations = generations

        self.rl = QLearningAgent()

        # mapping action -> operator
        self.mutations = [swap_mutation, pairwise_swap, inversion_mutation]
        self.crossovers = [one_point_crossover, two_point_crossover, uniform_crossover]

    # -------------------------------------------------
    def create_chromosome(self):
        chrom = []
        for t in range(self.inst.n_tasks):
            chrom += [t] * self.inst.subtasks_per_task[t]
        random.shuffle(chrom)
        return chrom

    def init_population(self):
        return [self.create_chromosome() for _ in range(self.pop_size)]

    def evaluate_population(self, pop):
        return [self.scheduler.evaluate(ind) for ind in pop]

    def tournament_selection(self, pop, fitness, k=3):
        selected = random.sample(range(len(pop)), k)
        best = min(selected, key=lambda i: fitness[i])
        return pop[best]

    # -------------------------------------------------
    def apply_action(self, action, p1, p2):
        # mutation actions
        if action <= 2:
            op = self.mutations[action]
            return op(p1.copy()), op(p2.copy())
        # crossover actions
        else:
            op = self.crossovers[action-3]
            return op(p1.copy(), p2.copy())

    # -------------------------------------------------
    def run(self):
        population = self.init_population()

        best_global = None
        best_fit = 1e18
        prev_best = 1e18

        state = 2  # start as stagnation

        for gen in range(self.generations):
            fitness = self.evaluate_population(population)
            elite_idx = np.argmin(fitness)
            elite = population[elite_idx]
            current_best = fitness[elite_idx]

            # ---------- RL state ----------
            improvement = prev_best - current_best
            next_state = self.rl.get_state(improvement)

            # ---------- choose operator ----------
            action = self.rl.choose_action(state)

            # ---------- GA reproduction ----------
            new_pop = [elite]

            while len(new_pop) < self.pop_size:
                p1 = self.tournament_selection(population, fitness)
                p2 = self.tournament_selection(population, fitness)

                c1, c2 = self.apply_action(action, p1, p2)
                new_pop += [c1, c2]

            population = new_pop[:self.pop_size]

            # ---------- reward ----------
            reward = prev_best - current_best
            if reward <= 0:
                reward = -5

            # ---------- update Q ----------
            self.rl.update(state, action, reward, next_state)

            state = next_state
            prev_best = current_best

            if current_best < best_fit:
                best_fit = current_best
                best_global = elite

            print(f"Gen {gen} | Best {best_fit} | Action {action}")

        return best_global, best_fit

##🧪 تست الگوریتم مقاله

In [ ]:
# module_content for instance_generator.py
module_content_instance = """import numpy as np
import random

class CMgInstance:
    def __init__(self, n_tasks=10, n_providers=3):
        self.n_tasks = n_tasks
        self.n_providers = n_providers

        # ---------- GENERATE STRUCTURE ----------
        self.subtasks_per_task = [random.randint(2,5) for _ in range(n_tasks)]
        self.machines_per_provider = [random.randint(2,5) for _ in range(n_providers)]

        # ---------- ARRIVAL TIMES ----------
        self.arrival_time = np.random.randint(0,200, size=n_tasks)

        # ---------- PROCESSING TIMES ----------
        # P[i][j][k][r]
        self.processing_time = self.generate_processing_times()

        # ---------- LOGISTICS ----------
        self.TSS = np.random.randint(5,30,(n_providers,n_providers))  # SP to SP
        self.TSC = np.random.randint(5,50,(n_providers,n_tasks))      # SP to customer

    # -----------------------------------------
    def generate_processing_times(self):
        P = {}
        for i in range(self.n_tasks):
            P[i] = {}
            for j in range(self.subtasks_per_task[i]):
                P[i][j] = {}
                for k in range(self.n_providers):
                    P[i][j][k] = {}
                    for r in range(self.machines_per_provider[k]):
                        P[i][j][k][r] = random.randint(10,100)
        return P

# ----------- helper to create benchmark sizes ----------
def generate_instance(size="small"):

    if size == "small":
        return CMgInstance(n_tasks=10, n_providers=3)
    elif size == "medium":
        return CMgInstance(n_tasks=20, n_providers=5)
    else:
        return CMgInstance(n_tasks=35, n_providers=8)
"""
with open('instance_generator.py', 'w') as f:
    f.write(module_content_instance)


# module_content for scheduler_simulator.py
module_content_scheduler = """import numpy as np
from collections import defaultdict

class Scheduler:
    def __init__(self, instance):
        self.inst = instance

    # --------------------------------------------------
    # انتخاب بهترین ماشین برای یک عملیات
    # --------------------------------------------------
    def select_best_machine(self, task, subtask, ready_time):
        best_finish = 1e18
        best_choice = None

        for k in range(self.inst.n_providers):
            for r in range(self.inst.machines_per_provider[k]):

                proc_time = self.inst.processing_time[task][subtask][k][r]
                finish_time = ready_time + proc_time

                if finish_time < best_finish:
                    best_finish = finish_time
                    best_choice = (k,r,proc_time)

        return best_choice  # provider, machine, processing_time

    # --------------------------------------------------
    # DECODE chromosome → schedule → makespan
    # --------------------------------------------------
    def evaluate(self, chromosome):

        # زمان آزاد شدن ماشین‌ها
        machine_available = defaultdict(int)

        # پایان آخرین subtask هر task
        task_last_finish = defaultdict(int)

        # شماره subtask بعدی که باید اجرا شود
        next_subtask_index = defaultdict(int)

        # شروع کار = arrival time
        for t in range(self.inst.n_tasks):
            task_last_finish[t] = self.inst.arrival_time[t]

        # ----------------------------------------------
        for gene in chromosome:
            task = gene

            subtask = next_subtask_index[task]

            # اگر همه subtasks انجام شده → رد کن
            if subtask >= self.inst.subtasks_per_task[task]:
                continue

            ready_time = task_last_finish[task]

            # انتخاب بهترین ماشین
            provider, machine, proc_time = self.select_best_machine(
                task, subtask, ready_time
            )

            machine_key = (provider, machine)
            start_time = max(machine_available[machine_key], ready_time)
            finish_time = start_time + proc_time

            # آپدیت وضعیت‌ها
            machine_available[machine_key] = finish_time
            task_last_finish[task] = finish_time
            next_subtask_index[task] += 1

        # ----------------------------------------------
        # makespan = آخرین پایان
        makespan = max(task_last_finish.values())
        return makespan
"""
with open('scheduler_simulator.py', 'w') as f:
    f.write(module_content_scheduler)


# module_content for operators.py
module_content_operators = """import random

# -----------------------------
# MUTATIONS
# -----------------------------
def swap_mutation(chrom):
    a, b = random.sample(range(len(chrom)), 2)
    chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

def pairwise_swap(chrom):
    for i in range(0, len(chrom)-1, 2):
        if random.random() < 0.5:
            chrom[i], chrom[i+1] = chrom[i+1], chrom[i]
    return chrom

def inversion_mutation(chrom):
    a, b = sorted(random.sample(range(len(chrom)), 2))
    chrom[a:b] = reversed(chrom[a:b])
    return chrom

# -----------------------------
# CROSSOVERS
# -----------------------------
def one_point_crossover(p1, p2):
    cut = random.randint(1, len(p1)-2)
    c1 = p1[:cut] + p2[cut:]
    c2 = p2[:cut] + p1[cut:]
    return c1, c2

def two_point_crossover(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    c1 = p1[:a] + p2[a:b] + p1[b:]
    c2 = p2[:a] + p1[a:b] + p2[b:]
    return c1, c2

def uniform_crossover(p1, p2):
    c1, c2 = [], []
    for g1, g2 in zip(p1, p2):
        if random.random() < 0.5:
            c1.append(g1); c2.append(g2)
        else:
            c1.append(g2); c2.append(g1)
    return c1, c2
"""
with open('operators.py', 'w') as f:
    f.write(module_content_operators)


# module_content for q_learning.py
module_content_q_learning = """import numpy as np
import random

class QLearningAgent:
    def __init__(self, n_states=3, n_actions=6,
                 alpha=0.1, gamma=0.9, epsilon=0.2):

        self.Q = np.zeros((n_states, n_actions))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    # ---------------------------------
    def get_state(self, improvement):
        if improvement > 20:
            return 0
        elif improvement > 0:
            return 1
        else:
            return 2

    # ---------------------------------
    def choose_action(self, state):
        # \u03b5-greedy
        if random.random() < self.epsilon:
            return random.randint(0,5)
        return np.argmax(self.Q[state])

    # ---------------------------------
    def update(self, state, action, reward, next_state):
        best_next = np.max(self.Q[next_state])

        self.Q[state, action] += self.alpha * (
            reward + self.gamma * best_next - self.Q[state, action]
        )"""

with open('q_learning.py', 'w') as f:
    f.write(module_content_q_learning)


# module_content for qlga.py
module_content_qlga = """import random
import numpy as np

from scheduler_simulator import Scheduler
from operators import *
from q_learning import QLearningAgent

class QLGA:
    def __init__(self, instance, pop_size=40, generations=60):
        self.inst = instance
        self.scheduler = Scheduler(instance)
        self.pop_size = pop_size
        self.generations = generations

        self.rl = QLearningAgent()

        # mapping action -> operator
        self.mutations = [swap_mutation, pairwise_swap, inversion_mutation]
        self.crossovers = [one_point_crossover, two_point_crossover, uniform_crossover]

    # -------------------------------------------------
    def create_chromosome(self):
        chrom = []
        for t in range(self.inst.n_tasks):
            chrom += [t] * self.inst.subtasks_per_task[t]
        random.shuffle(chrom)
        return chrom

    def init_population(self):
        return [self.create_chromosome() for _ in range(self.pop_size)]

    def evaluate_population(self, pop):
        return [self.scheduler.evaluate(ind) for ind in pop]

    def tournament_selection(self, pop, fitness, k=3):
        selected = random.sample(range(len(pop)), k)
        best = min(selected, key=lambda i: fitness[i])
        return pop[best]

    # -------------------------------------------------
    def apply_action(self, action, p1, p2):
        # mutation actions
        if action <= 2:
            op = self.mutations[action]
            return op(p1.copy()), op(p2.copy())
        # crossover actions
        else:
            op = self.crossovers[action-3]
            return op(p1.copy(), p2.copy())

    # -------------------------------------------------
    def run(self):
        population = self.init_population()

        best_global = None
        best_fit = 1e18
        prev_best = 1e18

        state = 2  # start as stagnation

        for gen in range(self.generations):
            fitness = self.evaluate_population(population)
            elite_idx = np.argmin(fitness)
            elite = population[elite_idx]
            current_best = fitness[elite_idx]

            # ---------- RL state ----------
            improvement = prev_best - current_best
            next_state = self.rl.get_state(improvement)

            # ---------- choose operator ----------
            action = self.rl.choose_action(state)

            # ---------- GA reproduction ----------
            new_pop = [elite]

            while len(new_pop) < self.pop_size:
                p1 = self.tournament_selection(population, fitness)
                p2 = self.tournament_selection(population, fitness)

                c1, c2 = self.apply_action(action, p1, p2)
                new_pop += [c1, c2]

            population = new_pop[:self.pop_size]

            # ---------- reward ----------
            reward = prev_best - current_best
            if reward <= 0:
                reward = -5

            # ---------- update Q ----------
            self.rl.update(state, action, reward, next_state)

            state = next_state
            prev_best = current_best

            if current_best < best_fit:
                best_fit = current_best
                best_global = elite

            print(f"Gen {gen} | Best {best_fit} | Action {action}")

        return best_global, best_fit
"""
with open('qlga.py', 'w') as f:
    f.write(module_content_qlga)


from instance_generator import generate_instance
from qlga import QLGA

inst = generate_instance("small")

qlga = QLGA(inst)
best_sol, best_fit = qlga.run()

print("\nFinal Makespan QLGA:", best_fit)

##📈 Step 6 — اجرای آزمایش‌ها و رسم نمودار (Reproducibility)



##📦 File Experiments.py

In [ ]:
import numpy as np
from instance_generator import generate_instance
from ga_baseline import GeneticAlgorithm
from qlga import QLGA
import matplotlib.pyplot as plt

def run_algorithm(algo_class, size, runs=5):
    results = []

    for r in range(runs):
        inst = generate_instance(size)
        algo = algo_class(inst)
        _, best_fit = algo.run()
        results.append(best_fit)

    return np.mean(results), np.std(results)

# --------------------------------------------
sizes = ["small", "medium", "large"]

ga_means = []
qlga_means = []

for s in sizes:
    print("\n===== SIZE:", s, "=====")

    ga_mean, _ = run_algorithm(GeneticAlgorithm, s)
    qlga_mean, _ = run_algorithm(QLGA, s)

    ga_means.append(ga_mean)
    qlga_means.append(qlga_mean)

# --------------------------------------------
# PLOT
plt.figure()
plt.plot(sizes, ga_means, marker='o', label="GA")
plt.plot(sizes, qlga_means, marker='o', label="QLGA")
plt.ylabel("Average Makespan")
plt.title("GA vs QLGA")
plt.legend()
plt.grid()

plt.savefig("result.png")
plt.show()